# Streaming

Streaming is a capability that allows AI providers to deliver responses as tokens are generated in real-time,
rather than waiting for the complete response to be finished.
You can visualize it as filling a bucket with water — streaming lets you watch the water level rising in real-time,
instead of placing the bucket somewhere, walking away, and only returning once it's completely full.

Streaming significantly improves the user experience,
as people can immediately see the response forming rather than staring at a loading indicator.
This is especially valuable since synchronous requests can take quite a long time to generate.

Let's implement streaming responses with Kotlin and Spring AI.
For this, we'll need to add dependencies.
In addition to `spring-ai-openai`, we'll need the Kotlin coroutines library to work with `Flow`.

In [1]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.10.1")

@file:DependsOn("org.springframework.ai:spring-ai-openai-spring-boot-starter:1.0.0-M6")

As in the previous notebooks, we'll need an API key.

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

Now let's create a `ChatModel` and pass in the `ChatOptions`:

In [3]:
import org.springframework.ai.chat.client.ChatClient
import org.springframework.ai.openai.OpenAiChatModel
import org.springframework.ai.openai.OpenAiChatOptions
import org.springframework.ai.openai.api.OpenAiApi

val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.7)
    .build()

val chatCompletion = OpenAiChatModel.builder()
    .openAiApi(openAiApi)
    .defaultOptions(openAiOptions)
    .build()

Now let's ask the model to generate a short fairy tale about rabbits:

In [4]:
import kotlinx.coroutines.reactive.asFlow

val response: Flow<String> = chatCompletion.stream("Generate me 10 sentences of a fairy tale about rabbits").asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time, in a lush green meadow filled with wildflowers, there lived a clever rabbit named Ruby. Ruby had big dreams of discovering the legendary Golden Carrot, said to grant wisdom to any rabbit who found it. One sunny morning, she gathered her friends—a shy bunny named Benny and a brave hare named Hazel—to embark on an adventure. They set off through the Whispering Woods, where the trees shared secrets of the past with the gentle breeze.

As they ventured deeper, they encountered a wise old tortoise named Tobias, who warned them of the tricky fox that guarded the Golden Carrot. Undeterred, Ruby devised a clever plan to outsmart the fox, using their unique talents to create a distraction. When the fox came to investigate, Benny hopped in one direction, while Hazel darted in the other, leaving the fox bewildered and chasing shadows. 

Finally, they reached the shimmering field where the Golden Carrot was said to grow, surrounded by sparkling fireflies that danced in the twilig

As you can see, the response starts printing immediately.
If this weren't a streaming response,
we would be waiting approximately 8 seconds (judging by the cell execution time) with nothing happening.

Now let's do the same thing using the `ChatClient`:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val response: Flow<String> = chatClient
    .prompt("Generate me 10 sentences of a fairy tale about rabbits")
    .stream()
    .content()
    .asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time, in a lush green meadow where wildflowers danced in the breeze, there lived a clever little rabbit named Oliver. Oliver had the softest fur and the brightest eyes, and he dreamed of finding the legendary Golden Carrot said to grant wishes to those pure of heart. One sunny morning, he gathered his friends—a brave rabbit named Bella and a wise old tortoise named Timothy—to embark on an adventure to find this magical treasure.

As they journeyed through the enchanted forest, they encountered a mischievous fox who tried to trick them into revealing their quest. But Oliver, quick on his feet, cleverly spun a tale about a hidden treasure guarded by a fierce dragon, which sent the fox scampering away in fear. The trio continued on their path, facing challenges like crossing a bubbling brook and climbing a steep hill, but they encouraged each other with laughter and songs.

After what felt like days of travel, they finally stumbled upon a sparkling glade where the Golden Carro